In [ ]:
!pip install --upgrade huggingface_hub
!pip install confluent_kafka
!pip install kafka-python

In [ ]:
from typing import Iterator

import torch

from model import get_input_token_length, run

DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\
"""
MAX_MAX_NEW_TOKENS = 2048
DEFAULT_MAX_NEW_TOKENS = 1024
MAX_INPUT_TOKEN_LENGTH = 4000

DESCRIPTION = """
# Llama-2 7B Chat

This Space demonstrates model [Llama-2-7b-chat](https://huggingface.co/meta-llama/Llama-2-7b-chat) by Meta, a Llama 2 model with 7B parameters fine-tuned for chat instructions. Feel free to play with it, or duplicate to run generations without a queue! If you want to run your own service, you can also [deploy the model on Inference Endpoints](https://huggingface.co/inference-endpoints).

🔎 For more details about the Llama 2 family of models and how to use them with `transformers`, take a look [at our blog post](https://huggingface.co/blog/llama2).

🔨 Looking for an even more powerful model? Check out the [13B version](https://huggingface.co/spaces/huggingface-projects/llama-2-13b-chat) or the large [70B model demo](https://huggingface.co/spaces/ysharma/Explore_llamav2_with_TGI).
"""

LICENSE = """
<p/>

---
As a derivate work of [Llama-2-7b-chat](https://huggingface.co/meta-llama/Llama-2-7b-chat) by Meta,
this demo is governed by the original [license](https://huggingface.co/spaces/huggingface-projects/llama-2-7b-chat/blob/main/LICENSE.txt) and [acceptable use policy](https://huggingface.co/spaces/huggingface-projects/llama-2-7b-chat/blob/main/USE_POLICY.md).
"""

if not torch.cuda.is_available():
    DESCRIPTION += '\n<p>Running on CPU 🥶 This demo does not work on CPU.</p>'


def clear_and_save_textbox(message = None):
    return '', message


def display_input(message = None,
                  history = None):
    history.append((message, ''))
    return history


def delete_prev_fn(history = None):
    try:
        message, _ = history.pop()
    except IndexError:
        message = ''
    return history, message or ''


def generate(
    message= None,
    history_with_input= None,
    system_prompt= None,
    max_new_tokens= None,
    temperature= None,
    top_p= None,
    top_k= None,
) :
    max_new_tokens = int(max_new_tokens)
    temperature =  float(temperature)
    top_p = float(top_p)
    top_k =  int(top_k)
    
    if max_new_tokens > MAX_MAX_NEW_TOKENS:
        raise ValueError

    history = history_with_input[:-1]
    generator = run(message, history, system_prompt, max_new_tokens, temperature, top_p, top_k)
    
    try:
        first_response = next(generator)
        yield history + [(message, first_response)]
    except StopIteration:
        yield history + [(message, '')]
    for response in generator:
        yield history + [(message, response)]


def process_example(message= None) :
    generator = generate(message, [], DEFAULT_SYSTEM_PROMPT, 1024, 1, 0.95, 50)
    for x in generator:
        pass
    return '', x

def inference(input= None) :
    generator = generate(*input)
    for x in generator:
        pass
    return '', x

def check_input_token_length(message= None, chat_history= None, system_prompt= None) :
    input_token_length = get_input_token_length(message, chat_history, system_prompt)
    if input_token_length > MAX_INPUT_TOKEN_LENGTH:
        raise gr.Error(f'The accumulated input is too long ({input_token_length} > {MAX_INPUT_TOKEN_LENGTH}). Clear your chat history and try again.')

#result = process_example(message= 'What is iPaaS (Integration Platform as a Service)')    
#print(type(result))
#print(result)

In [ ]:
def test_inference(input= None) :
    return ('', [('What is iPaaS (Integration Platform as a Service)', " Hello! I'm here to help you with your question. iPaaS (Integration Platform as a Service) is a cloud-based platform that enables organizations to integrate their applications, data, and services with ease. It provides a centralized and standardized integration environment that enables organizations to connect their applications, APIs, and data sources, and automate the flow of data and processes across these systems.\niPaaS platforms offer a variety of benefits, including:\n1. Easy integration: iPaaS platforms provide a simple and intuitive interface for integrating applications, data, and services, without the need for complex coding or programming.\n2. Scalability: iPaaS platforms are designed to handle large volumes of data and traffic, making them scalable to meet the needs of growing organizations.\n3. Flexibility: iPaaS platforms offer a wide range of integration options, including real-time, batch, and file-based integration, as well as support for a variety of protocols and formats.\n4. Security: iPaaS platforms provide robust security features, such as encryption, authentication, and access control, to ensure the integrity and security of data during integration.\n5. Low cost: iPaaS platforms are typically offered on a subscription basis, making them a cost-effective solution for organizations of all sizes.\n\nOverall, iPaaS platforms provide a powerful and flexible solution for organizations looking to integrate their applications, data, and services in the cloud. However, it's important to note that not all iPaaS platforms are created equal, and organizations should carefully evaluate their options before selecting a platform that best meets their needs.\nI hope this helps! If you have any further questions, please don't hesitate to ask.")])

In [6]:
%run kafka.ipynb
group = 'answer-llama2-7b-1'
topic = 'llama2-7b-01'
name  = 'llama-2-7b-chat'

In [13]:
#get_last_n_messages(1,topic=topic,group_id='test-consumer')[0]['answer']  

In [ ]:
def process_instruction(input,id=None):
    dt1 = datetime.now()
    result = inference(input=input)
    respone = result[1][len(result[1])-1][1]
    dt2 = datetime.now()
    time = (dt2 - dt1).seconds
    answer = create_answer_message (answer=str(respone),id=id,time=time)
    send_to_topic(answer,topic)
    #print(answer)
    
def wait_for_instruction_and_process(consumer,topic=None,key='instruction',process=None):
    running = True
    while running:
        msg = consumer.poll(120.0)
        if not msg or  msg.error():
            continue
                
        dict_str = msg.value().decode('utf-8')
        message = ast.literal_eval(dict_str)
        #print(key, message.keys())
        if key in message.keys():
            id = message['id']
            input = ast.literal_eval(message[key])
            #print(id)
            process(input,id=id)
            #running = False
            #break  
        elif 'action' in message.keys():
            running = False
            break   

In [ ]:
# job start    
job_topic='jobs'
msg = create_status_message(type='backend',status='start',name=name,topic=topic)
send_to_topic(msg,job_topic)
id = msg['id']            
    
# job    
consumer = create_and_subscribe_consumer(group = group,topic = topic)
wait_for_instruction_and_process(consumer,topic=topic,key='instruction',process=process_instruction)

# job end   
msg = create_status_message(id=id,type='backend',status='end',name=name,topic=topic)
send_to_topic(msg,job_topic)